In [1]:
!mkdir data
!curl https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet --output data/fhv_tripdata_2021-01.parquet
!curl https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet --output data/fhv_tripdata_2021-02.parquet

mkdir: cannot create directory ‘data’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.3M  100 11.3M    0     0  18.9M      0 --:--:-- --:--:-- --:--:-- 18.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.1M  100 10.1M    0     0  20.6M      0 --:--:-- --:--:-- --:--:-- 20.6M


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [4]:
file_path = 'data/fhv_tripdata_2021-01.parquet'
df = pd.read_parquet(file_path)
df.count()

dispatching_base_num      1154112
pickup_datetime           1154112
dropOff_datetime          1154112
PUlocationID               195845
DOlocationID               991892
SR_Flag                         0
Affiliated_base_number    1153227
dtype: int64

In [5]:
df['duration'] = (df['dropOff_datetime'] - df['pickup_datetime']).apply(lambda delta: delta.total_seconds()/60)
df['duration'].describe()

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
25%      7.766667e+00
50%      1.340000e+01
75%      2.228333e+01
max      4.233710e+05
Name: duration, dtype: float64

In [6]:
df = df[(df['duration'] <=60) & (df['duration']>=1)]

In [7]:
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID'] = df['DOlocationID'].fillna(-1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
(df[df['PUlocationID'] == -1].count()['PUlocationID'])/df.count()['PUlocationID']

0.8352732770722617

In [9]:
categorical = ['PUlocationID', 'DOlocationID']
df[categorical] = df[categorical].astype(str)
train_dicts = df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [10]:
X_train.shape

(1109826, 525)

In [11]:
target = 'duration'
y_train = df[target]

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [12]:
mean_squared_error(y_train, y_pred, squared=False)

10.528519107210744

In [13]:
def load_data(file_path):
  df = pd.read_parquet(file_path)
  df['duration'] = (df['dropOff_datetime'] - df['pickup_datetime']).apply(lambda delta: delta.total_seconds()/60)
  df = df[(df['duration'] <=60) & (df['duration']>=1)]
  df['PUlocationID'] = df['PUlocationID'].fillna(-1)
  df['DOlocationID'] = df['DOlocationID'].fillna(-1)
  (df[df['PUlocationID'] == -1].count()['PUlocationID'])/df.count()['PUlocationID']
  return df

In [14]:
df_train = load_data('data/fhv_tripdata_2021-01.parquet')
df_val = load_data('data/fhv_tripdata_2021-02.parquet')

In [15]:
target = 'duration'
categorical = ['PUlocationID', 'DOlocationID']

In [16]:
df_train[categorical] = df_train[categorical].astype(str)
train_dicts = df_train[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df_train[target]

In [17]:
df_val[categorical] = df_val[categorical].astype(str)
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_val[target]

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)

In [19]:
mean_squared_error(y_val, y_pred, squared=False)

11.014283196111764